<a href="https://colab.research.google.com/github/DOMINION-JOHN1/skin-lesion-detection/blob/main/cutler_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Specify the directory path
drive_mount_point = '/content/drive/MyDrive/'

# Check if the directory exists, and create it if it doesn't
if not os.path.exists(drive_mount_point):
    os.makedirs(drive_mount_point)

# Now, mount Google Drive to the specified directory
from google.colab import drive
drive.mount(drive_mount_point, force_remount=True)


Mounted at /content/drive/MyDrive/


In [ ]:
!git clone --recursive https://github.com/facebookresearch/CutLER
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!wget http://dl.fbaipublicfiles.com/cutler/checkpoints/cutler_cascade_final.pth

Cloning into 'CutLER'...
remote: Enumerating objects: 581, done.
remote: Counting objects: 100% (424/424), done.
remote: Compressing objects: 100% (333/333), done.
remote: Total 581 (delta 115), reused 366 (delta 89), pack-reused 157
Receiving objects: 100% (581/581), 44.84 MiB | 38.82 MiB/s, done.
Resolving deltas: 100% (161/161), done.
Submodule 'third_party/TokenCut' (https://github.com/YangtaoWANG95/TokenCut.git) registered for path 'third_party/TokenCut'
Cloning into '/content/CutLER/cutler/demo/CutLER/third_party/TokenCut'...
remote: Enumerating objects: 212, done.        
remote: Counting objects: 100% (62/62), done.        
remote: Compressing objects: 100% (38/38), done.        
remote: Total 212 (delta 42), reused 32 (delta 23), pack-reused 150        
Receiving objects: 100% (212/212), 7.29 MiB | 12.28 MiB/s, done.
Resolving deltas: 100% (81/81), done.
Submodule path 'third_party/TokenCut': checked out '5eba77f56f0fc92ffc79589d0cbb119aab5f9b2d'
  Cloning https://github.com/f

In [ ]:
%cd CutLER/

/content/CutLER/cutler/demo/CutLER


In [ ]:
!pip install git+https://github.com/cocodataset/panopticapi.git
!pip install git+https://github.com/mcordts/cityscapesScripts.git
!pip install -r requirements.txt

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-t07aiyw0
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-t07aiyw0
  Resolved https://github.com/cocodataset/panopticapi.git to commit 7bb4655548f98f3fedc07bf37e9040a992b054b0
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/mcordts/cityscapesScripts.git to /tmp/pip-req-build-9rk7kx86
  Running command git clone --filter=blob:none --quiet https://github.com/mcordts/cityscapesScripts.git /tmp/pip-req-build-9rk7kx86
  Resolved https://github.com/mcordts/cityscapesScripts.git to commit a7ac7b4062d1a80ed5e22d2ea2179c886801c77d
  Preparing metadata (setup.py) ... done
  Using cached submitit-1.5.1-py3-none-any.whl (74 kB)
  Using cached faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (85.5 MB)
  Using cached opencv_python-4.6.0.66-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.9 MB

In [ ]:
%cd cutler/demo

/content/CutLER/cutler/demo/CutLER/cutler/demo


In [ ]:
import argparse
import multiprocessing as mp
import numpy as np
import os
import tempfile
import time
import cv2
import matplotlib.pyplot as plt

from detectron2.config import get_cfg
from detectron2.data.detection_utils import read_image
from detectron2.utils.logger import setup_logger
import sys
sys.path.append('./')
sys.path.append('../')
from config import add_cutler_config
from predictor import VisualizationDemo

In [ ]:
# ----------------------- Dataset registration -----------------------
def register_dataset(name, json_path, img_dir):
    if name not in DatasetCatalog.list():
        register_coco_instances(name, {}, json_path, img_dir)
    else:
        print(f"Dataset {name} is already registered.")

In [ ]:
# Debug: List all registered datasets to ensure correct registration
def list_registered_datasets():
    print("Registered datasets:")
    for dataset_name in DatasetCatalog.list():
        print(f"- {dataset_name}")

In [ ]:

import json
import os
import uuid
from glob import glob
from tqdm import tqdm
from detectron2.structures import BoxMode
def yolo_to_coco(image_dir, label_dir, output_path, dataset_type):
    images = []
    annotations = []
    # Define category mappings
    categories = [{'id': 0, 'name': 'unlabeled'}]  # Replace with your actual category names
    annotation_id = 1
    # Get all image files
    image_files = glob(os.path.join(image_dir, '*.png'))
    for image_file in tqdm(image_files, desc="Processing images"):
        # Generate a random UUID for the image ID
        image_id = uuid.uuid4()
        image_info = {
            'file_name': os.path.basename(image_file),
            'height': 640,
            'width': 640,
            'id': str(image_id)  # Convert the UUID to a string
        }
        images.append(image_info)
        label_file = os.path.join(label_dir, os.path.basename(image_file).replace('.png', '.txt'))
        with open(label_file, 'r') as file:
            for line in file:
                class_id, x_center, y_center, width, height = [float(x) for x in line.split()]
                # Convert from YOLO to absolute COCO format
                x_min = (x_center - width / 2) * 640
                y_min = (y_center - height / 2) * 640
                x_max = (x_center + width / 2) * 640
                y_max = (y_center + height / 2) * 640
                # width_abs = width * 640
                # height_abs = height * 640
                annotation = {
                    'id': annotation_id,
                    'image_id': str(image_id),  # Convert the UUID to a string
                    'category_id': int(class_id),
                    # 'bbox': [x_min, y_min, width_abs, height_abs],
                    'bbox': [x_min, y_min, x_max - x_min, y_max - y_min],#
                    'bbox_mode': BoxMode.XYXY_ABS,
                    # 'area': width_abs * height_abs,
                    'iscrowd': 0
                }
                annotations.append(annotation)
                annotation_id += 1
    coco_format = {
        'images': images,
        'annotations': annotations,
        'categories': categories
    }
    with open(output_path, 'w') as output_file:
        json.dump(coco_format, output_file)

In [ ]:
yolo_to_coco("/content/drive/MyDrive/MyDrive/skin lesions/images/train",'/content/drive/MyDrive/MyDrive/skin lesions/labels/train',  "/content/drive/MyDrive/MyDrive/skin lesions/labels/train/annotations_train.json","train" )

Processing images: 100%|██████████| 7/7 [00:02<00:00,  2.65it/s]


In [ ]:
import json
import os
import uuid
from glob import glob
from tqdm import tqdm
from detectron2.structures import BoxMode
def yolo_to_coco(image_dir, label_dir, output_path, dataset_type):
    images = []
    annotations = []
    # Define category mappings
    categories = [{'id': 0, 'name': 'unlabeled'}]  # Replace with your actual category names
    annotation_id = 1
    # Get all image files
    image_files = glob(os.path.join(image_dir, '*.png'))
    for image_file in tqdm(image_files, desc="Processing images"):
        # Generate a random UUID for the image ID
        image_id = uuid.uuid4()
        image_info = {
            'file_name': os.path.basename(image_file),
            'height': 640,
            'width': 640,
            'id': str(image_id)  # Convert the UUID to a string
        }
        images.append(image_info)
        label_file = os.path.join(label_dir, os.path.basename(image_file).replace('.png', '.txt'))
        with open(label_file, 'r') as file:
            for line in file:
                class_id, x_center, y_center, width, height = [float(x) for x in line.split()]
                # Convert from YOLO to absolute COCO format
                x_min = (x_center - width / 2) * 640
                y_min = (y_center - height / 2) * 640
                x_max = (x_center + width / 2) * 640
                y_max = (y_center + height / 2) * 640
                # width_abs = width * 640
                # height_abs = height * 640
                annotation = {
                    'id': annotation_id,
                    'image_id': str(image_id),  # Convert the UUID to a string
                    'category_id': int(class_id),
                    # 'bbox': [x_min, y_min, width_abs, height_abs],
                    'bbox': [x_min, y_min, x_max - x_min, y_max - y_min],#
                    'bbox_mode': BoxMode.XYXY_ABS,
                    # 'area': width_abs * height_abs,
                    'iscrowd': 0
                }
                annotations.append(annotation)
                annotation_id += 1
    coco_format = {
        'images': images,
        'annotations': annotations,
        'categories': categories
    }
    with open(output_path, 'w') as output_file:
        json.dump(coco_format, output_file)





In [ ]:
yolo_to_coco("/content/drive/MyDrive/MyDrive/skin lesions/images/val",'/content/drive/MyDrive/MyDrive/skin lesions/labels/val',  "/content/drive/MyDrive/MyDrive/skin lesions/labels/train/annotations_val.json","val" )

Processing images: 100%|██████████| 5/5 [00:01<00:00,  2.74it/s]


In [ ]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader, DatasetCatalog, MetadataCatalog
import torch  # Import torch for checking CUDA availability


In [ ]:
register_dataset("train", "/content/drive/MyDrive/MyDrive/skin lesions/labels/train/annotations_train.json", "/content/drive/MyDrive/MyDrive/skin lesions/images/train")
register_dataset("val", "/content/drive/MyDrive/MyDrive/skin lesions/labels/val/annotations_val.json", "/content/drive/MyDrive/MyDrive/skin lesions/images/val")
list_registered_datasets()  # Call to list registered datasets for debugging

Registered datasets:
- coco_2014_train
- coco_2014_val
- coco_2014_minival
- coco_2014_valminusminival
- coco_2017_train
- coco_2017_val
- coco_2017_test
- coco_2017_test-dev
- coco_2017_val_100
- keypoints_coco_2014_train
- keypoints_coco_2014_val
- keypoints_coco_2014_minival
- keypoints_coco_2014_valminusminival
- keypoints_coco_2017_train
- keypoints_coco_2017_val
- keypoints_coco_2017_val_100
- coco_2017_train_panoptic_separated
- coco_2017_train_panoptic_stuffonly
- coco_2017_train_panoptic
- coco_2017_val_panoptic_separated
- coco_2017_val_panoptic_stuffonly
- coco_2017_val_panoptic
- coco_2017_val_100_panoptic_separated
- coco_2017_val_100_panoptic_stuffonly
- coco_2017_val_100_panoptic
- lvis_v1_train
- lvis_v1_val
- lvis_v1_test_dev
- lvis_v1_test_challenge
- lvis_v0.5_train
- lvis_v0.5_val
- lvis_v0.5_val_rand_100
- lvis_v0.5_test
- lvis_v0.5_train_cocofied
- lvis_v0.5_val_cocofied
- cityscapes_fine_instance_seg_train
- cityscapes_fine_sem_seg_train
- cityscapes_fine_instanc

In [ ]:
from detectron2.modeling.roi_heads import ROI_HEADS_REGISTRY

In [ ]:
from detectron2.modeling.roi_heads import ROIHeads

In [ ]:
#from detectron2.modeling.roi_heads import ROI_HEADS_REGISTRY
#from CutLER.cutler.modeling.roi_heads.custom_cascade_rcnn.py import CustomCascadeROIHeads

# Register the custom ROI heads with a unique name
#ROI_HEADS_REGISTRY.register("CustomCascadeROIHeads", lambda: CustomCascadeROIHeads)

In [ ]:

import inspect
import logging
import numpy as np
from typing import Dict, List, Optional, Tuple
import torch
from torch import nn

from detectron2.config import configurable
from detectron2.layers import ShapeSpec, nonzero_tuple
from detectron2.structures import Boxes, pairwise_iou
from structures import pairwise_iou_max_scores
from detectron2.structures import ImageList, Instances
from detectron2.utils.events import get_event_storage
from detectron2.utils.registry import Registry

from detectron2.modeling.backbone.resnet import BottleneckBlock, ResNet
from detectron2.modeling.matcher import Matcher
from detectron2.modeling.poolers import ROIPooler
from detectron2.modeling.proposal_generator.proposal_utils import add_ground_truth_to_proposals
from detectron2.modeling.sampling import subsample_labels
from detectron2.modeling.roi_heads.box_head import build_box_head
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers
from detectron2.modeling.roi_heads.keypoint_head import build_keypoint_head
from detectron2.modeling.roi_heads.mask_head import build_mask_head

from detectron2.modeling.box_regression import Box2BoxTransform
import torch.nn.functional as F
from colored import fg


if "CustomStandardROIHeads" not in ROI_HEADS_REGISTRY:
    @ROI_HEADS_REGISTRY.register()
    class CustomStandardROIHeads(ROIHeads):
        """
        It's "standard" in a sense that there is no ROI transform sharing
        or feature sharing between tasks.
        Each head independently processes the input features by each head's
        own pooler and head.

        This class is used by most models, such as FPN and C5.
        To implement more models, you can subclass it and implement a different
        :meth:`forward()` or a head.
        """

        @configurable
        def __init__(
            self,
            *,
            box_in_features: List[str],
            box_pooler: ROIPooler,
            box_head: nn.Module,
            box_predictor: nn.Module,
            mask_in_features: Optional[List[str]] = None,
            mask_pooler: Optional[ROIPooler] = None,
            mask_head: Optional[nn.Module] = None,
            keypoint_in_features: Optional[List[str]] = None,
            keypoint_pooler: Optional[ROIPooler] = None,
            keypoint_head: Optional[nn.Module] = None,
            train_on_pred_boxes: bool = False,
            box2box_transform = Box2BoxTransform,
            use_droploss: bool = False,
            droploss_iou_thresh: float = 1.0,
            **kwargs,
        ):
            """
            NOTE: this interface is experimental.

            Args:
                box_in_features (list[str]): list of feature names to use for the box head.
                box_pooler (ROIPooler): pooler to extra region features for box head
                box_head (nn.Module): transform features to make box predictions
                box_predictor (nn.Module): make box predictions from the feature.
                    Should have the same interface as :class:`FastRCNNOutputLayers`.
                mask_in_features (list[str]): list of feature names to use for the mask
                    pooler or mask head. None if not using mask head.
                mask_pooler (ROIPooler): pooler to extract region features from image features.
                    The mask head will then take region features to make predictions.
                    If None, the mask head will directly take the dict of image features
                    defined by `mask_in_features`
                mask_head (nn.Module): transform features to make mask predictions
                keypoint_in_features, keypoint_pooler, keypoint_head: similar to ``mask_*``.
                train_on_pred_boxes (bool): whether to use proposal boxes or
                    predicted boxes from the box head to train other heads.
            """
            super().__init__(**kwargs)
            # keep self.in_features for backward compatibility
            self.in_features = self.box_in_features = box_in_features
            self.box_pooler = box_pooler
            self.box_head = box_head
            self.box_predictor = box_predictor

            self.mask_on = mask_in_features is not None
            if self.mask_on:
                self.mask_in_features = mask_in_features
                self.mask_pooler = mask_pooler
                self.mask_head = mask_head

            self.keypoint_on = keypoint_in_features is not None
            if self.keypoint_on:
                self.keypoint_in_features = keypoint_in_features
                self.keypoint_pooler = keypoint_pooler
                self.keypoint_head = keypoint_head

            self.train_on_pred_boxes = train_on_pred_boxes
            self.use_droploss = use_droploss
            self.box2box_transform = box2box_transform
            self.droploss_iou_thresh = droploss_iou_thresh

        @classmethod
        def from_config(cls, cfg, input_shape):
            ret = super().from_config(cfg)
            ret["train_on_pred_boxes"] = cfg.MODEL.ROI_BOX_HEAD.TRAIN_ON_PRED_BOXES
            # Subclasses that have not been updated to use from_config style construction
            # may have overridden _init_*_head methods. In this case, those overridden methods
            # will not be classmethods and we need to avoid trying to call them here.
            # We test for this with ismethod which only returns True for bound methods of cls.
            # Such subclasses will need to handle calling their overridden _init_*_head methods.
            if cfg.MODEL.ROI_HEADS.USE_DROPLOSS:
                ret['use_droploss'] = True
                ret['droploss_iou_thresh'] = cfg.MODEL.ROI_HEADS.DROPLOSS_IOU_THRESH
                ret['box2box_transform'] = Box2BoxTransform(weights=cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS)
            if inspect.ismethod(cls._init_box_head):
                ret.update(cls._init_box_head(cfg, input_shape))
            if inspect.ismethod(cls._init_mask_head):
                ret.update(cls._init_mask_head(cfg, input_shape))
            if inspect.ismethod(cls._init_keypoint_head):
                ret.update(cls._init_keypoint_head(cfg, input_shape))
            return ret

        @classmethod
        def _init_box_head(cls, cfg, input_shape):
            # fmt: off
            in_features       = cfg.MODEL.ROI_HEADS.IN_FEATURES
            pooler_resolution = cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION
            pooler_scales     = tuple(1.0 / input_shape[k].stride for k in in_features)
            sampling_ratio    = cfg.MODEL.ROI_BOX_HEAD.POOLER_SAMPLING_RATIO
            pooler_type       = cfg.MODEL.ROI_BOX_HEAD.POOLER_TYPE
            # fmt: on

            # If CustomStandardROIHeads is applied on multiple feature maps (as in FPN),
            # then we share the same predictors and therefore the channel counts must be the same
            in_channels = [input_shape[f].channels for f in in_features]
            # Check all channel counts are equal
            assert len(set(in_channels)) == 1, in_channels
            in_channels = in_channels[0]

            box_pooler = ROIPooler(
                output_size=pooler_resolution,
                scales=pooler_scales,
                sampling_ratio=sampling_ratio,
                pooler_type=pooler_type,
            )
            # Here we split "box head" and "box predictor", which is mainly due to historical reasons.
            # They are used together so the "box predictor" layers should be part of the "box head".
            # New subclasses of ROIHeads do not need "box predictor"s.
            box_head = build_box_head(
                cfg, ShapeSpec(channels=in_channels, height=pooler_resolution, width=pooler_resolution)
            )
            box_predictor = FastRCNNOutputLayers(cfg, box_head.output_shape)
            return {
                "box_in_features": in_features,
                "box_pooler": box_pooler,
                "box_head": box_head,
                "box_predictor": box_predictor,
            }

        @classmethod
        def _init_mask_head(cls, cfg, input_shape):
            if not cfg.MODEL.MASK_ON:
                return {}
            # fmt: off
            in_features       = cfg.MODEL.ROI_HEADS.IN_FEATURES
            pooler_resolution = cfg.MODEL.ROI_MASK_HEAD.POOLER_RESOLUTION
            pooler_scales     = tuple(1.0 / input_shape[k].stride for k in in_features)
            sampling_ratio    = cfg.MODEL.ROI_MASK_HEAD.POOLER_SAMPLING_RATIO
            pooler_type       = cfg.MODEL.ROI_MASK_HEAD.POOLER_TYPE
            # fmt: on

            in_channels = [input_shape[f].channels for f in in_features][0]

            ret = {"mask_in_features": in_features}
            ret["mask_pooler"] = (
                ROIPooler(
                    output_size=pooler_resolution,
                    scales=pooler_scales,
                    sampling_ratio=sampling_ratio,
                    pooler_type=pooler_type,
                )
                if pooler_type
                else None
            )
            if pooler_type:
                shape = ShapeSpec(
                    channels=in_channels, width=pooler_resolution, height=pooler_resolution
                )
            else:
                shape = {f: input_shape[f] for f in in_features}
            ret["mask_head"] = build_mask_head(cfg, shape)
            return ret

        @classmethod
        def _init_keypoint_head(cls, cfg, input_shape):
            if not cfg.MODEL.KEYPOINT_ON:
                return {}
            # fmt: off
            in_features       = cfg.MODEL.ROI_HEADS.IN_FEATURES
            pooler_resolution = cfg.MODEL.ROI_KEYPOINT_HEAD.POOLER_RESOLUTION
            pooler_scales     = tuple(1.0 / input_shape[k].stride for k in in_features)  # noqa
            sampling_ratio    = cfg.MODEL.ROI_KEYPOINT_HEAD.POOLER_SAMPLING_RATIO
            pooler_type       = cfg.MODEL.ROI_KEYPOINT_HEAD.POOLER_TYPE
            # fmt: on

            in_channels = [input_shape[f].channels for f in in_features][0]

            ret = {"keypoint_in_features": in_features}
            ret["keypoint_pooler"] = (
                ROIPooler(
                    output_size=pooler_resolution,
                    scales=pooler_scales,
                    sampling_ratio=sampling_ratio,
                    pooler_type=pooler_type,
                )
                if pooler_type
                else None
            )
            if pooler_type:
                shape = ShapeSpec(
                    channels=in_channels, width=pooler_resolution, height=pooler_resolution
                )
            else:
                shape = {f: input_shape[f] for f in in_features}
            ret["keypoint_head"] = build_keypoint_head(cfg, shape)
            return ret

        def forward(
            self,
            images: ImageList,
            features: Dict[str, torch.Tensor],
            proposals: List[Instances],
            targets: Optional[List[Instances]] = None,
        ) -> Tuple[List[Instances], Dict[str, torch.Tensor]]:
            """
            See :class:`ROIHeads.forward`.
            """
            del images
            if self.training:
                assert targets, "'targets' argument is required during training"
                proposals = self.label_and_sample_proposals(proposals, targets)
            del targets

            if self.training:
                losses = self._forward_box(features, proposals)
                # Usually the original proposals used by the box head are used by the mask, keypoint
                # heads. But when `self.train_on_pred_boxes is True`, proposals will contain boxes
                # predicted by the box head.
                losses.update(self._forward_mask(features, proposals))
                losses.update(self._forward_keypoint(features, proposals))
                return proposals, losses
            else:
                pred_instances = self._forward_box(features, proposals)
                # During inference cascaded prediction is used: the mask and keypoints heads are only
                # applied to the top scoring box detections.
                pred_instances = self.forward_with_given_boxes(features, pred_instances)
                return pred_instances, {}

        def forward_with_given_boxes(
            self, features: Dict[str, torch.Tensor], instances: List[Instances]
        ) -> List[Instances]:
            """
            Use the given boxes in `instances` to produce other (non-box) per-ROI outputs.

            This is useful for downstream tasks where a box is known, but need to obtain
            other attributes (outputs of other heads).
            Test-time augmentation also uses this.

            Args:
                features: same as in `forward()`
                instances (list[Instances]): instances to predict other outputs. Expect the keys
                    "pred_boxes" and "pred_classes" to exist.

            Returns:
                list[Instances]:
                    the same `Instances` objects, with extra
                    fields such as `pred_masks` or `pred_keypoints`.
            """
            assert not self.training
            assert instances[0].has("pred_boxes") and instances[0].has("pred_classes")

            instances = self._forward_mask(features, instances)
            instances = self._forward_keypoint(features, instances)
            return instances

        def _forward_box(self, features: Dict[str, torch.Tensor], proposals: List[Instances]):
            """
            Forward logic of the box prediction branch. If `self.train_on_pred_boxes is True`,
                the function puts predicted boxes in the `proposal_boxes` field of `proposals` argument.

            Args:
                features (dict[str, Tensor]): mapping from feature map names to tensor.
                    Same as in :meth:`ROIHeads.forward`.
                proposals (list[Instances]): the per-image object proposals with
                    their matching ground truth.
                    Each has fields "proposal_boxes", and "objectness_logits",
                    "gt_classes", "gt_boxes".

            Returns:
                In training, a dict of losses.
                In inference, a list of `Instances`, the predicted instances.
            """
            features = [features[f] for f in self.box_in_features]
            box_features = self.box_pooler(features, [x.proposal_boxes for x in proposals]) # torch.Size([512 * batch_size, 256, 7, 7])
            box_features = self.box_head(box_features) # torch.Size([512 * batch_size, 1024])
            predictions = self.box_predictor(box_features) # [torch.Size([512 * batch_size, 2]), torch.Size([512 * batch_size, 4])]

            no_gt_found = False
            if self.use_droploss and self.training:
                # the first K proposals are GT proposals
                try:
                    box_num_list = [len(x.gt_boxes) for x in proposals]
                    gt_num_list = [torch.unique(x.gt_boxes.tensor[:100], dim=0).size()[0] for x in proposals]
                except:
                    box_num_list = [0 for _ in proposals]
                    gt_num_list = [0 for _ in proposals]
                    no_gt_found = True

            if self.use_droploss and self.training and not no_gt_found:
                # NOTE: maximum overlapping with GT (IoU)
                predictions_delta = predictions[1]
                proposal_boxes = Boxes.cat([x.proposal_boxes for x in proposals])
                predictions_bbox = self.box2box_transform.apply_deltas(predictions_delta, proposal_boxes.tensor)
                idx_start = 0
                iou_max_list = []
                for idx, x in enumerate(proposals):
                    idx_end = idx_start + box_num_list[idx]
                    iou_max_list.append(pairwise_iou_max_scores(predictions_bbox[idx_start:idx_end], x.gt_boxes[:gt_num_list[idx]].tensor))
                    idx_start = idx_end
                iou_max = torch.cat(iou_max_list, dim=0)

            del box_features

            if self.training:
                if self.use_droploss and not no_gt_found:
                    weights = iou_max.le(self.droploss_iou_thresh).float()
                    weights = 1 - weights.ge(1.0).float()
                    losses = self.box_predictor.losses(predictions, proposals, weights=weights.detach())
                else:
                    losses = self.box_predictor.losses(predictions, proposals)
                if self.train_on_pred_boxes: # default is false
                    with torch.no_grad():
                        pred_boxes = self.box_predictor.predict_boxes_for_gt_classes(
                            predictions, proposals
                        )
                        for proposals_per_image, pred_boxes_per_image in zip(proposals, pred_boxes):
                            proposals_per_image.proposal_boxes = Boxes(pred_boxes_per_image)
                return losses
            else:
                pred_instances, _ = self.box_predictor.inference(predictions, proposals)
                return pred_instances

        def _forward_mask(self, features: Dict[str, torch.Tensor], instances: List[Instances]):
            """
            Forward logic of the mask prediction branch.

            Args:
                features (dict[str, Tensor]): mapping from feature map names to tensor.
                    Same as in :meth:`ROIHeads.forward`.
                instances (list[Instances]): the per-image instances to train/predict masks.
                    In training, they can be the proposals.
                    In inference, they can be the boxes predicted by R-CNN box head.

            Returns:
                In training, a dict of losses.
                In inference, update `instances` with new fields "pred_masks" and return it.
            """
            if not self.mask_on:
                return {} if self.training else instances

            if self.training:
                # head is only trained on positive proposals.
                instances, _ = select_foreground_proposals(instances, self.num_classes)

            if self.mask_pooler is not None:
                features = [features[f] for f in self.mask_in_features]
                boxes = [x.proposal_boxes if self.training else x.pred_boxes for x in instances]
                features = self.mask_pooler(features, boxes)
            else:
                features = {f: features[f] for f in self.mask_in_features}
            return self.mask_head(features, instances)

        def _forward_keypoint(self, features: Dict[str, torch.Tensor], instances: List[Instances]):
            """
            Forward logic of the keypoint prediction branch.

            Args:
                features (dict[str, Tensor]): mapping from feature map names to tensor.
                    Same as in :meth:`ROIHeads.forward`.
                instances (list[Instances]): the per-image instances to train/predict keypoints.
                    In training, they can be the proposals.
                    In inference, they can be the boxes predicted by R-CNN box head.

            Returns:
                In training, a dict of losses.
                In inference, update `instances` with new fields "pred_keypoints" and return it.
            """
            if not self.keypoint_on:
                return {} if self.training else instances

            if self.training:
                # head is only trained on positive proposals with >=1 visible keypoints.
                instances, _ = select_foreground_proposals(instances, self.num_classes)
                instances = select_proposals_with_visible_keypoints(instances)

            if self.keypoint_pooler is not None:
                features = [features[f] for f in self.keypoint_in_features]
                boxes = [x.proposal_boxes if self.training else x.pred_boxes for x in instances]
                features = self.keypoint_pooler(features, boxes)
            else:
                features = {f: features[f] for f in self.keypoint_in_features}
            return self.keypoint_head(features, instances)





In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# Modified by XuDong Wang from https://github.com/facebookresearch/detectron2/blob/main/detectron2/modeling/roi_heads/cascade_rcnn.py

from typing import List
import torch
from torch import nn
from torch.autograd.function import Function

from detectron2.config import configurable
from detectron2.layers import ShapeSpec
from detectron2.structures import Boxes, pairwise_iou
from structures import pairwise_iou_max_scores
from detectron2.structures import Instances
from detectron2.utils.events import get_event_storage

from detectron2.modeling.box_regression import Box2BoxTransform
from detectron2.modeling.matcher import Matcher
from detectron2.modeling.poolers import ROIPooler
from detectron2.modeling.roi_heads.box_head import build_box_head
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, fast_rcnn_inference
from detectron2.modeling.roi_heads import ROI_HEADS_REGISTRY

import torch.nn.functional as F




class _ScaleGradient(Function):
    @staticmethod
    def forward(ctx, input, scale):
        ctx.scale = scale
        return input

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output * ctx.scale, None

if "CustomCascadeROIHeads"  not in ROI_HEADS_REGISTRY:
    @ROI_HEADS_REGISTRY.register()
    class CustomCascadeROIHeads(CustomStandardROIHeads):
        """
        The ROI heads that implement :paper:`Cascade R-CNN`.
        """

        @configurable
        def __init__(
            self,
            *,
            box_in_features: List[str],
            box_pooler: ROIPooler,
            box_heads: List[nn.Module],
            box_predictors: List[nn.Module],
            proposal_matchers: List[Matcher],
            **kwargs,
        ):
            """
            NOTE: this interface is experimental.

            Args:
                box_pooler (ROIPooler): pooler that extracts region features from given boxes
                box_heads (list[nn.Module]): box head for each cascade stage
                box_predictors (list[nn.Module]): box predictor for each cascade stage
                proposal_matchers (list[Matcher]): matcher with different IoU thresholds to
                    match boxes with ground truth for each stage. The first matcher matches
                    RPN proposals with ground truth, the other matchers use boxes predicted
                    by the previous stage as proposals and match them with ground truth.
            """
            assert "proposal_matcher" not in kwargs, (
                "CustomCascadeROIHeads takes 'proposal_matchers=' for each stage instead "
                "of one 'proposal_matcher='."
            )
            # The first matcher matches RPN proposals with ground truth, done in the base class
            kwargs["proposal_matcher"] = proposal_matchers[0]
            num_stages = self.num_cascade_stages = len(box_heads)
            box_heads = nn.ModuleList(box_heads)
            box_predictors = nn.ModuleList(box_predictors)
            assert len(box_predictors) == num_stages, f"{len(box_predictors)} != {num_stages}!"
            assert len(proposal_matchers) == num_stages, f"{len(proposal_matchers)} != {num_stages}!"
            super().__init__(
                box_in_features=box_in_features,
                box_pooler=box_pooler,
                box_head=box_heads,
                box_predictor=box_predictors,
                **kwargs,
            )
            self.proposal_matchers = proposal_matchers

        @classmethod
        def from_config(cls, cfg, input_shape):
            ret = super().from_config(cfg, input_shape)
            ret.pop("proposal_matcher")
            return ret

        @classmethod
        def _init_box_head(cls, cfg, input_shape):
            # fmt: off
            in_features              = cfg.MODEL.ROI_HEADS.IN_FEATURES
            pooler_resolution        = cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION
            pooler_scales            = tuple(1.0 / input_shape[k].stride for k in in_features)
            sampling_ratio           = cfg.MODEL.ROI_BOX_HEAD.POOLER_SAMPLING_RATIO
            pooler_type              = cfg.MODEL.ROI_BOX_HEAD.POOLER_TYPE
            cascade_bbox_reg_weights = cfg.MODEL.ROI_BOX_CASCADE_HEAD.BBOX_REG_WEIGHTS
            cascade_ious             = cfg.MODEL.ROI_BOX_CASCADE_HEAD.IOUS
            assert len(cascade_bbox_reg_weights) == len(cascade_ious)
            assert cfg.MODEL.ROI_BOX_HEAD.CLS_AGNOSTIC_BBOX_REG,  \
                "CustomCascadeROIHeads only support class-agnostic regression now!"
            assert cascade_ious[0] == cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS[0]
            # fmt: on

            in_channels = [input_shape[f].channels for f in in_features]
            # Check all channel counts are equal
            assert len(set(in_channels)) == 1, in_channels
            in_channels = in_channels[0]

            box_pooler = ROIPooler(
                output_size=pooler_resolution,
                scales=pooler_scales,
                sampling_ratio=sampling_ratio,
                pooler_type=pooler_type,
            )
            pooled_shape = ShapeSpec(
                channels=in_channels, width=pooler_resolution, height=pooler_resolution
            )

            box_heads, box_predictors, proposal_matchers = [], [], []
            for match_iou, bbox_reg_weights in zip(cascade_ious, cascade_bbox_reg_weights):
                box_head = build_box_head(cfg, pooled_shape)
                box_heads.append(box_head)
                box_predictors.append(
                    FastRCNNOutputLayers(
                        cfg,
                        box_head.output_shape,
                        box2box_transform=Box2BoxTransform(weights=bbox_reg_weights),
                    )
                )
                proposal_matchers.append(Matcher([match_iou], [0, 1], allow_low_quality_matches=False))
            return {
                "box_in_features": in_features,
                "box_pooler": box_pooler,
                "box_heads": box_heads,
                "box_predictors": box_predictors,
                "proposal_matchers": proposal_matchers,
            }

    def forward(self, images, features, proposals, targets=None):
        del images
        if self.training:
            proposals = self.label_and_sample_proposals(proposals, targets)

        if self.training:
            # Need targets to box head
            losses = self._forward_box(features, proposals, targets)
            losses.update(self._forward_mask(features, proposals))
            losses.update(self._forward_keypoint(features, proposals))
            return proposals, losses
        else:
            pred_instances = self._forward_box(features, proposals)
            pred_instances = self.forward_with_given_boxes(features, pred_instances)
            return pred_instances, {}

    def _forward_box(self, features, proposals, targets=None):
        """
        Args:
            features, targets: the same as in
                Same as in :meth:`ROIHeads.forward`.
            proposals (list[Instances]): the per-image object proposals with
                their matching ground truth.
                Each has fields "proposal_boxes", and "objectness_logits",
                "gt_classes", "gt_boxes".
        """
        features = [features[f] for f in self.box_in_features]
        head_outputs = []  # (predictor, predictions, proposals)
        prev_pred_boxes = None
        image_sizes = [x.image_size for x in proposals]
        for k in range(self.num_cascade_stages):
            if k > 0:
                # The output boxes of the previous stage are used to create the input
                # proposals of the next stage.
                proposals = self._create_proposals_from_boxes(prev_pred_boxes, image_sizes)
                if self.training:
                    proposals = self._match_and_label_boxes(proposals, k, targets)
            predictions = self._run_stage(features, proposals, k)
            prev_pred_boxes = self.box_predictor[k].predict_boxes(predictions, proposals)
            head_outputs.append((self.box_predictor[k], predictions, proposals))

        no_gt_found = False
        if self.training:
            losses = {}
            storage = get_event_storage()
            for stage, (predictor, predictions, proposals) in enumerate(head_outputs):
                no_gt_found = False
                with storage.name_scope("stage{}".format(stage)):
                    if self.use_droploss:
                        try:
                            box_num_list = [len(x.gt_boxes) for x in proposals]
                            gt_num_list = [torch.unique(x.gt_boxes.tensor[:100], dim=0).size()[0] for x in proposals]
                        except:
                            box_num_list = [0 for x in proposals]
                            gt_num_list = [0 for x in proposals]
                            no_gt_found = True

                        if not no_gt_found:
                            # NOTE: confidence score
                            prediction_score, predictions_delta = predictions[0], predictions[1]
                            prediction_score = F.softmax(prediction_score, dim=1)[:,0]

                            # NOTE: maximum overlapping with GT (IoU)
                            proposal_boxes = Boxes.cat([x.proposal_boxes for x in proposals])
                            predictions_bbox = predictor.box2box_transform.apply_deltas(predictions_delta, proposal_boxes.tensor)
                            idx_start = 0
                            iou_max_list = []
                            for idx, x in enumerate(proposals):
                                idx_end = idx_start + box_num_list[idx]
                                iou_max_list.append(pairwise_iou_max_scores(predictions_bbox[idx_start:idx_end], x.gt_boxes[:gt_num_list[idx]].tensor))
                                idx_start = idx_end
                            iou_max = torch.cat(iou_max_list, dim=0)

                            # NOTE: get the weight of each proposal
                            weights = iou_max.le(self.droploss_iou_thresh).float()
                            weights = 1 - weights.ge(1.0).float()
                            stage_losses = predictor.losses(predictions, proposals, weights=weights.detach())
                        else:
                            stage_losses = predictor.losses(predictions, proposals)
                    else:
                        stage_losses = predictor.losses(predictions, proposals)
                losses.update({k + "_stage{}".format(stage): v for k, v in stage_losses.items()})
            return losses
        else:
            # Each is a list[Tensor] of length #image. Each tensor is Ri x (K+1)
            scores_per_stage = [h[0].predict_probs(h[1], h[2]) for h in head_outputs]

            # Average the scores across heads
            scores = [
                sum(list(scores_per_image)) * (1.0 / self.num_cascade_stages)
                for scores_per_image in zip(*scores_per_stage)
            ]
            # Use the boxes of the last head
            predictor, predictions, proposals = head_outputs[-1]
            boxes = predictor.predict_boxes(predictions, proposals)
            pred_instances, _ = fast_rcnn_inference(
                boxes,
                scores,
                image_sizes,
                predictor.test_score_thresh,
                predictor.test_nms_thresh,
                predictor.test_topk_per_image,
            )
            return pred_instances

    @torch.no_grad()
    def _match_and_label_boxes(self, proposals, stage, targets):
        """
        Match proposals with groundtruth using the matcher at the given stage.
        Label the proposals as foreground or background based on the match.

        Args:
            proposals (list[Instances]): One Instances for each image, with
                the field "proposal_boxes".
            stage (int): the current stage
            targets (list[Instances]): the ground truth instances

        Returns:
            list[Instances]: the same proposals, but with fields "gt_classes" and "gt_boxes"
        """
        num_fg_samples, num_bg_samples = [], []
        for proposals_per_image, targets_per_image in zip(proposals, targets):
            match_quality_matrix = pairwise_iou(
                targets_per_image.gt_boxes, proposals_per_image.proposal_boxes
            )
            # proposal_labels are 0 or 1
            matched_idxs, proposal_labels = self.proposal_matchers[stage](match_quality_matrix)
            if len(targets_per_image) > 0:
                gt_classes = targets_per_image.gt_classes[matched_idxs]
                # Label unmatched proposals (0 label from matcher) as background (label=num_classes)
                gt_classes[proposal_labels == 0] = self.num_classes
                gt_boxes = targets_per_image.gt_boxes[matched_idxs]
            else:
                gt_classes = torch.zeros_like(matched_idxs) + self.num_classes
                gt_boxes = Boxes(
                    targets_per_image.gt_boxes.tensor.new_zeros((len(proposals_per_image), 4))
                )
            proposals_per_image.gt_classes = gt_classes
            proposals_per_image.gt_boxes = gt_boxes

            num_fg_samples.append((proposal_labels == 1).sum().item())
            num_bg_samples.append(proposal_labels.numel() - num_fg_samples[-1])

        # Log the number of fg/bg samples in each stage
        storage = get_event_storage()
        storage.put_scalar(
            "stage{}/roi_head/num_fg_samples".format(stage),
            sum(num_fg_samples) / len(num_fg_samples),
        )
        storage.put_scalar(
            "stage{}/roi_head/num_bg_samples".format(stage),
            sum(num_bg_samples) / len(num_bg_samples),
        )
        return proposals

    def _run_stage(self, features, proposals, stage):
        """
        Args:
            features (list[Tensor]): #lvl input features to ROIHeads
            proposals (list[Instances]): #image Instances, with the field "proposal_boxes"
            stage (int): the current stage

        Returns:
            Same output as `FastRCNNOutputLayers.forward()`.
        """
        box_features = self.box_pooler(features, [x.proposal_boxes for x in proposals])
        # The original implementation averages the losses among heads,
        # but scale up the parameter gradients of the heads.
        # This is equivalent to adding the losses among heads,
        # but scale down the gradients on features.
        if self.training:
            box_features = _ScaleGradient.apply(box_features, 1.0 / self.num_cascade_stages)
        box_features = self.box_head[stage](box_features)
        return self.box_predictor[stage](box_features)

    def _create_proposals_from_boxes(self, boxes, image_sizes):
        """
        Args:
            boxes (list[Tensor]): per-image predicted boxes, each of shape Ri x 4
            image_sizes (list[tuple]): list of image shapes in (h, w)

        Returns:
            list[Instances]: per-image proposals with the given boxes.
        """
        # Just like RPN, the proposals should not have gradients
        boxes = [Boxes(b.detach()) for b in boxes]
        proposals = []
        for boxes_per_image, image_size in zip(boxes, image_sizes):
            boxes_per_image.clip(image_size)
            if self.training:
                # do not filter empty boxes at inference time,
                # because the scores from each stage need to be aligned and added later
                boxes_per_image = boxes_per_image[boxes_per_image.nonempty()]
            prop = Instances(image_size)
            prop.proposal_boxes = boxes_per_image
            proposals.append(prop)
        return proposals


In [ ]:

cfg = get_cfg()
add_cutler_config(cfg)
cfg.merge_from_file('/content/CutLER/cutler/model_zoo/configs/COCO-Semisupervised/cascade_mask_rcnn_R_50_FPN_100perc.yaml')
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("val",)
cfg.MODEL.ROI_HEADS.NAME ="CustomCascadeROIHeads"
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = "http://dl.fbaipublicfiles.com/cutler/checkpoints/cutler_cascade_final.pth"
cfg.INPUT.MIN_SIZE_TRAIN = (640, )
cfg.INPUT.MAX_SIZE_TRAIN = 640
cfg.INPUT.MIN_SIZE_TEST = 640
cfg.INPUT.MAX_SIZE_TEST = 640
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = [25, 50]
cfg.SOLVER.WARMUP_ITERS = 10
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.SEED = 10000
cfg.OUTPUT_DIR = './train1'
cfg.OUTPUT_ENABLED = True
cfg.MODEL.DEVICE == "cpu" #and cfg.MODEL.RESNETS.NORM == 'SyncBN':
#cfg.MODEL.RESNETS.NORM = "BN"
#cfg.MODEL.FPN.NORM = "BN"
# (Optional) Modify loss function if needed (refer to Detectron2 documentation)

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[04/02 09:18:50 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(
      256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): SyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (fpn_output2): Conv2d(
      256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): SyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (fpn_lateral3): Conv2d(
      512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): SyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (fpn_output3): Conv2d(
      256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): SyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (fpn_lateral4): Conv2d(
      1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): SyncBatchNorm

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
cutler_cascade_final.pth: 575MB [00:02, 256MB/s]                           


[04/02 09:18:53 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


ERROR [04/02 09:19:03 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/detectron2/engine/train_loop.py", line 155, in train
    self.run_step()
  File "/usr/local/lib/python3.10/dist-packages/detectron2/engine/defaults.py", line 496, in run_step
    self._trainer.run_step()
  File "/usr/local/lib/python3.10/dist-packages/detectron2/engine/train_loop.py", line 310, in run_step
    loss_dict = self.model(data)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/detectron2/modeling/meta_arch/rcnn.py", line 167, in forward
    _, detector_losses = self.roi_heads(images, features, proposals, gt_instances)
  File "/usr/lo

NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [ ]:
from detectron2.modeling.roi_heads import ROI_HEADS_REGISTRY
from CutLER.cutler.modeling.roi_heads.custom_cascade_rcnn.py import CustomCascadeROIHeads

# Register the custom ROI heads with a unique name
ROI_HEADS_REGISTRY.register("custom_cascade_roi_heads", lambda: CustomCascadeROIHeads)


In [ ]:
ROI_HEADS_REGISTRY.register("CustomCascadeROIHeads")